In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
from openpyxl import load_workbook

# 源Excel文件路径
source_file_path = '../forhx/对账明细查询表_20240701_17时06分.xlsx'
# 目标Excel文件路径
target_file_path = '../forhx/对账单2024.06月.xlsx'

# 使用pandas读取Excel文件的元数据，但不加载数据
xls = pd.ExcelFile(target_file_path,engine='openpyxl')
workbook=load_workbook(filename=target_file_path)
# 获取所有工作表的名称
sheet_names = xls.sheet_names

sourcefile=pd.read_excel(source_file_path,sheet_name=0) #会自动关闭文件
print('====================================info=======================================')
#print(sourcefile.info())
print('===============================================================================')
        
        
def new_columnOrder(supplynamevalue,df):
    new_order = ['变动时间', '变动说明', '银行','常规回款','现金核销回款','非现金核销回款','现金核销订单金额','非现金核销订单金额','常规订单金额','调换货回款','换货订单金额']  # 新的列顺序
    df = df[new_order].reset_index(drop=True)  # 重新排序列 df.reset_index(drop=True)
    
   
    # 特定列插入固定值
    df.insert(loc=2,column='保证金',value='')
    df.insert(loc=3,column='工行',value='') 
    df.insert(loc=4,column='招行',value='')
    df.insert(loc=5,column='订单结案-现金',value='')
    df.insert(loc=6,column='订单结案-现金核销',value='')
    df.insert(loc=7,column='订单结案-非现金核销',value='')
    df.insert(loc=8,column='核销-现金核销',value='')
    df.insert(loc=9,column='核销-非现金核销',value='')
    df.insert(loc=10,column='出库-现金核销',value='') 
    df.insert(loc=11,column='出库-非现金核销',value='')
    df.insert(loc=12,column='出库-现金',value='')
    df.insert(loc=13,column='出库-调换货',value='')
    
    for lineindex in range(df.shape[0]):
        if df.loc[lineindex,'银行']=='工行':
            df.loc[lineindex,'工行']=df.loc[lineindex, '常规回款']
            df.loc[lineindex,'招行']=''
        elif df.loc[lineindex,'银行']=='招行':
            df.loc[lineindex,'工行']=''
            df.loc[lineindex,'招行']=df.loc[lineindex, '常规回款']   
        else:
            print(supplynamevalue+" 既不是招行，也不是工行，将会赋值为空")
            df.loc[lineindex,'工行']=''
            df.loc[lineindex,'招行']=''
    for lineindex in range(df.shape[0]):
        if df.loc[lineindex,'变动说明']=='回款方式其他':
            df.loc[lineindex,'订单结案-现金']=df.loc[lineindex,'常规回款']
        elif df.loc[lineindex,'变动说明']=='回款方式核销回款-现金核销':
            df.loc[lineindex,'核销-现金核销']=df.loc[lineindex,'现金核销回款']
           
        elif df.loc[lineindex,'变动说明']=='回款方式核销回款-非现金核销': 
            df.loc[lineindex,'核销-非现金核销']=df.loc[lineindex,'非现金核销回款']
        elif df.loc[lineindex,'变动说明']=='核销订单':
          
            df.loc[lineindex,'出库-现金核销']=df.loc[lineindex,'现金核销订单金额']
            df.loc[lineindex,'出库-非现金核销']=df.loc[lineindex,'非现金核销订单金额']
        elif  df.loc[lineindex,'变动说明']=='正常订单-账余常规':
            df.loc[lineindex,'出库-现金']=df.loc[lineindex,'常规订单金额']
        elif df.loc[lineindex,'变动说明']=='退货单': 
            df.loc[lineindex,'出库-调换货']=df.loc[lineindex,'调换货回款']
        elif df.loc[lineindex,'变动说明']=='换货订单-账余常规':
            df.loc[lineindex,'出库-调换货']=df.loc[lineindex,'换货订单金额']
        else:
            print('变动说明列='+df.loc[lineindex,'变动说明']+' 不在预先设定的值（回款方式其他,回款方式核销回款-现金核销,正常订单-账余常规...）跳过不处理')
            
            
            
            
    # 定义一个列名列表，列出你想要删除的列
    columns_to_delete = ['银行','常规回款','现金核销回款','非现金核销回款','现金核销订单金额','非现金核销订单金额','常规订单金额','调换货回款','换货订单金额']

    # 循环遍历列名列表，逐一删除列
    for column in columns_to_delete:
        if column in df.columns:
            del df[column]
        else:
            print(f"Column {column} not found in DataFrame.")
            break
    return df      
    
def handexcelSheet(sheetname):
    # 通过工作表名称读取
    subsheetinfo = xls.parse(sheetname)

    try:
        supplynamekey = subsheetinfo.iloc[0, 0]
        supplynamevalue = subsheetinfo.iloc[0, 1]
        print(supplynamekey,supplynamevalue)
    except KeyError:
        print('the sheet:'+sheetname+'is not my target sheet, please next.')
        return
        
    if supplynamekey=='供应商名称：':
        
        # 选取代理商名称列是supplynamevalue的行
        filtered_rows = sourcefile[sourcefile['代理商名称'] == supplynamevalue]
        if filtered_rows.shape[0]==0:
            print(source_file_path+' 文件中没有找到 '+str(supplynamevalue)+' 对应的信息')
        else:
            df_order=new_columnOrder(supplynamevalue,filtered_rows)
            print("=========================将要拷贝到目标文件的内容========================\n")
            print(df_order)
            print('===========================================================================')
            print('写文件前大小为 '+str(os.path.getsize(target_file_path)))
            '''
            # 追加df_order到Excel文件的特定工作表中
            with pd.ExcelWriter(target_file_path, engine='openpyxl', mode='a') as writer:  # 使用'a'模式追加写入   
                df_order.to_excel(writer, sheet_name=sheetname, index=False, header=False)  # 设置header=False避免重复写入列名
            '''
            
            sheet=workbook[sheetname]
            #确定插入位置
            insert_row_position = 7
            # 执行插入操作
            sheet.insert_rows(insert_row_position, amount=df_order.shape[0])
            # 填充数据
            '''
            for idx, row_data in enumerate(df_order, start=insert_row_position):
                for col_idx, cell_value in enumerate(row_data, start=1):
                    sheet.cell(row=idx+1, column=col_idx, value=cell_value)
            '''
            
            
            
            for i in range(df_order.shape[0]):
               
                for j in range(df_order.shape[1]):
                    print(insert_row_position+i,j+1,df_order.iat[i,j])
                    sheet.cell(row=insert_row_position+i,column=j+1,value=df_order.iat[i,j])
            
            
            
            
            print('写文件后大小为 '+str(os.path.getsize(target_file_path)))
    else:
        print ('read next sheet') 
    
    
#handexcelSheet('福州芮时利')


count=0
# 打印所有工作表名称
for name in sheet_names:
    count+=1
    print('一共有'+str(len(sheet_names))+'个工作表，正在处理第'+str(count)+'个工作表->'+name)
    if name in {'结束合作', 'Sheet1', 'Sheet2'}:
        pass
    else:
        handexcelSheet(name)



workbook.save(filename=target_file_path)
        




====================================info=======================================
一共有262个工作表，正在处理第1个工作表->本月期初（原）
序号 代理商
read next sheet
一共有262个工作表，正在处理第2个工作表->本月期初
品牌 序号
read next sheet
一共有262个工作表，正在处理第3个工作表->本月发生（护肤）
品牌 序号
read next sheet
一共有262个工作表，正在处理第4个工作表->本月发生（洗护）
品牌 序号
read next sheet
一共有262个工作表，正在处理第5个工作表->护肤核对-余额保证金
品牌 序号
read next sheet
一共有262个工作表，正在处理第6个工作表->洗护核对-余额保证金
品牌 序号
read next sheet
一共有262个工作表，正在处理第7个工作表->河北瑞俪美
供应商名称： 河北瑞俪美企业管理咨询有限公司
河北瑞俪美企业管理咨询有限公司 既不是招行，也不是工行，将会赋值为空
=========================将要拷贝到目标文件的内容========================

                 变动时间    变动说明 保证金 工行 招行  订单结案-现金 订单结案-现金核销 订单结案-非现金核销 核销-现金核销  \
0 2024-06-06 09:42:00  回款方式其他                  0                                

  核销-非现金核销 出库-现金核销 出库-非现金核销 出库-现金 出库-调换货  
0                                         
写文件前大小为 2124869
7 1 2024-06-06 09:42:00
7 2 回款方式其他
7 3 
7 4 
7 5 
7 6 0
7 7 
7 8 
7 9 
7 10 
7 11 
7 12 
7 13 
7 14 
写文件后大小为 2124869
一共有262个工作表，正在处理第8个工作表->成都盛泽西
供应商名称： 成都市盛泽西企业管理咨询有限公司
../forhx/对账

7 1 2024-06-07 11:56:00
7 2 换货订单-账余常规
7 3 
7 4 
7 5 
7 6 
7 7 
7 8 
7 9 
7 10 
7 11 
7 12 
7 13 
7 14 14868.0
8 1 2024-06-11 09:42:00
8 2 换货订单-账余常规
8 3 
8 4 
8 5 
8 6 
8 7 
8 8 
8 9 
8 10 
8 11 
8 12 
8 13 
8 14 11885.56
9 1 2024-06-25 13:19:00
9 2 换货订单-账余常规
9 3 
9 4 
9 5 
9 6 
9 7 
9 8 
9 9 
9 10 
9 11 
9 12 
9 13 
9 14 0.0
10 1 2024-06-27 14:00:00
10 2 回款方式核销回款-现金核销
10 3 
10 4 
10 5 
10 6 
10 7 
10 8 
10 9 8950.0
10 10 
10 11 
10 12 
10 13 
10 14 
11 1 2024-06-06 09:45:00
11 2 回款方式其他
11 3 
11 4 
11 5 
11 6 0.0
11 7 
11 8 
11 9 
11 10 
11 11 
11 12 
11 13 
11 14 
12 1 2024-06-18 16:16:00
12 2 回款方式网上转账
12 3 
12 4 4485.0
12 5 
12 6 
12 7 
12 8 
12 9 
12 10 
12 11 
12 12 
12 13 
12 14 
13 1 2024-06-06 13:40:00
13 2 退货单
13 3 
13 4 
13 5 
13 6 
13 7 
13 8 
13 9 
13 10 
13 11 
13 12 
13 13 
13 14 -14710.0
14 1 2024-06-06 13:41:00
14 2 退货单
14 3 
14 4 
14 5 
14 6 
14 7 
14 8 
14 9 
14 10 
14 11 
14 12 
14 13 
14 14 -12206.36
15 1 2024-06-12 16:24:00
15 2 退货单
15 3 
15 4 
15 5 
15 6 
15 7 
15 8

供应商名称： 河南美共体商贸有限公司
../forhx/对账明细查询表_20240701_17时06分.xlsx 文件中没有找到 河南美共体商贸有限公司 对应的信息
一共有262个工作表，正在处理第17个工作表->侯马市永盛
供应商名称： 侯马市永盛和化妆品经销部
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
侯马市永盛和化妆品经销部 既不是招行，也不是工行，将会赋值为空
变动说明列=回款方式网上转账 不在预先设定的值（回款方式其他,回款方式核销回款-现金核销,正常订单-账余常规...）跳过不处理
=========================将要拷贝到目标文件的内容========================

                  变动时间       变动说明 保证金     工行 招行 订单结案-现金 订单结案-现金核销 订单结案-非现金核销  \
0  2024-06-17 14:19:00  换货订单-账余常规                                              
1  2

7 1 2024-06-26 13:44:00
7 2 回款方式核销回款-非现金核销
7 3 
7 4 
7 5 
7 6 
7 7 
7 8 
7 9 
7 10 49398.0
7 11 
7 12 
7 13 
7 14 
8 1 2024-06-26 13:45:00
8 2 回款方式核销回款-现金核销
8 3 
8 4 
8 5 
8 6 
8 7 
8 8 
8 9 330.0
8 10 
8 11 
8 12 
8 13 
8 14 
9 1 2024-06-12 16:56:00
9 2 回款方式其他
9 3 
9 4 
9 5 
9 6 0.0
9 7 
9 8 
9 9 
9 10 
9 11 
9 12 
9 13 
9 14 
10 1 2024-06-06 14:54:00
10 2 正常订单-老店季度政策款-常规
10 3 
10 4 
10 5 
10 6 
10 7 
10 8 
10 9 
10 10 
10 11 
10 12 
10 13 
10 14 
11 1 2024-06-04 16:00:00
11 2 正常订单-老店季度政策款-常规
11 3 
11 4 
11 5 
11 6 
11 7 
11 8 
11 9 
11 10 
11 11 
11 12 
11 13 
11 14 
12 1 2024-06-04 16:01:00
12 2 正常订单-老店季度政策款-常规
12 3 
12 4 
12 5 
12 6 
12 7 
12 8 
12 9 
12 10 
12 11 
12 12 
12 13 
12 14 
13 1 2024-06-26 13:10:00
13 2 正常订单-老店季度政策款-常规
13 3 
13 4 
13 5 
13 6 
13 7 
13 8 
13 9 
13 10 
13 11 
13 12 
13 13 
13 14 
14 1 2024-06-14 14:05:00
14 2 正常订单-账余常规
14 3 
14 4 
14 5 
14 6 
14 7 
14 8 
14 9 
14 10 
14 11 
14 12 
14 13 0.0
14 14 
15 1 2024-06-25 11:49:00
15 2 正常订单-账余常规
15 3 
15 4 
15 5 


供应商名称： 兰州凯诺商贸有限公司
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
兰州凯诺商贸有限公司 既不是招行，也不是工行，将会赋值为空
=========================将要拷贝到目标文件的内容========================

                  变动时间       变动说明 保证金 工行 招行 订单结案-现金 订单结案-现金核销 订单结案-非现金核销  \
0  2024-06-06 09:43:00     回款方式其他                 0                        
1  2024-06-04 11:20:00  正常订单-账余常规                                          
2  2024-06-14 14:06:00  正常订单-账余常规                                          
3  2024-06-26 13:24:00  正常订单-账余常规                                          
4  2024-06-26 13:25:00  正常订单-账余常规                                          
5  2024-06-26 13:26:00  正常订单-账余常规                                          
6  2024-06-26 13:26:00  正常订单-账余常规                        

...

7 1 2024-06-12 16:06:00
7 2 核销订单
7 3 
7 4 
7 5 
7 6 
7 7 
7 8 
7 9 
7 10 
7 11 23230.8
7 12 0.0
7 13 
7 14 
8 1 2024-06-12 16:06:00
8 2 核销订单
8 3 
8 4 
8 5 
8 6 
8 7 
8 8 
8 9 
8 10 
8 11 0.0
8 12 0.0
8 13 
8 14 
9 1 2024-06-18 14:39:00
9 2 换货订单-账余常规
9 3 
9 4 
9 5 
9 6 
9 7 
9 8 
9 9 
9 10 
9 11 
9 12 
9 13 
9 14 26084.56
10 1 2024-06-26 13:51:00
10 2 回款方式核销回款-非现金核销
10 3 
10 4 
10 5 
10 6 
10 7 
10 8 
10 9 
10 10 391994.62
10 11 
10 12 
10 13 
10 14 
11 1 2024-06-26 13:53:00
11 2 回款方式核销回款-现金核销
11 3 
11 4 
11 5 
11 6 
11 7 
11 8 
11 9 3110.0
11 10 
11 11 
11 12 
11 13 
11 14 
12 1 2024-06-27 13:59:00
12 2 回款方式核销回款-现金核销
12 3 
12 4 
12 5 
12 6 
12 7 
12 8 
12 9 3372.4
12 10 
12 11 
12 12 
12 13 
12 14 
13 1 2024-06-11 16:15:00
13 2 回款方式其他
13 3 
13 4 
13 5 
13 6 0.0
13 7 
13 8 
13 9 
13 10 
13 11 
13 12 
13 13 
13 14 
14 1 2024-06-14 09:52:00
14 2 回款方式其他
14 3 
14 4 
14 5 
14 6 648.0
14 7 
14 8 
14 9 
14 10 
14 11 
14 12 
14 13 
14 14 
15 1 2024-06-18 14:41:00
15 2 退货单
15 3 
15 4 
15 5 
15 6

供应商名称： 郑州芮合雅化妆品有限公司X
../forhx/对账明细查询表_20240701_17时06分.xlsx 文件中没有找到 郑州芮合雅化妆品有限公司X 对应的信息
一共有262个工作表，正在处理第254个工作表->福建金黔X
供应商名称： 福建金黔贸易有限公司X
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
福建金黔贸易有限公司X 既不是招行，也不是工行，将会赋值为空
变动说明列=回款方式网上转账 不在预先设定的值（回款方式其他,回款方式核销回款-现金核销,正常订单-账余常规...）跳过不处理
变动说明列=回款方式网上转账 不在预先设定的值（回款方式其他,回款方式核销回款-现金核销,正常订单-账余常规...）跳过不处理
=========================将要拷贝到目标文件的内容========================

                 变动时间       变动说明 保证金       工行 招行 订单结案-现金 订单结案-现金核销 订单结案-非现金核销  \
0 2024-06-24 10:07:00   回款方式网上转账      20010.9                                   
1 2024-06-24 10:37:00   回款方式网上转账            0                                   
2 2024-06-21 15:43:00  正常订单-账余常规                                                
3 2024-06-21 15:48:00  正常订单-账余常规                                                
4 2024-06-21 15:47:

7 1 2024-06-20 13:49:00
7 2 核销订单
7 3 
7 4 
7 5 
7 6 
7 7 
7 8 
7 9 
7 10 
7 11 0.0
7 12 13608.0
7 13 
7 14 
8 1 2024-06-20 16:23:00
8 2 回款方式核销回款-非现金核销
8 3 
8 4 
8 5 
8 6 
8 7 
8 8 
8 9 
8 10 31.37
8 11 
8 12 
8 13 
8 14 
9 1 2024-06-20 16:22:00
9 2 回款方式核销回款-现金核销
9 3 
9 4 
9 5 
9 6 
9 7 
9 8 
9 9 5.07
9 10 
9 11 
9 12 
9 13 
9 14 
10 1 2024-06-20 16:24:00
10 2 回款方式结案核销回款-非现金核销
10 3 
10 4 
10 5 
10 6 
10 7 
10 8 
10 9 
10 10 
10 11 
10 12 
10 13 
10 14 
11 1 2024-06-20 16:23:00
11 2 回款方式其他
11 3 
11 4 
11 5 
11 6 388.74
11 7 
11 8 
11 9 
11 10 
11 11 
11 12 
11 13 
11 14 
12 1 2024-06-20 16:24:00
12 2 回款方式其他
12 3 
12 4 
12 5 
12 6 6300.0
12 7 
12 8 
12 9 
12 10 
12 11 
12 12 
12 13 
12 14 
13 1 2024-06-28 15:17:00
13 2 回款方式其他
13 3 
13 4 
13 5 
13 6 18540.0
13 7 
13 8 
13 9 
13 10 
13 11 
13 12 
13 13 
13 14 
14 1 2024-06-29 00:00:00
14 2 回款方式网上转账
14 3 
14 4 
14 5 30010.0
14 6 
14 7 
14 8 
14 9 
14 10 
14 11 
14 12 
14 13 
14 14 
15 1 2024-06-29 00:00:00
15 2 回款方式网上转账
15 3 
15 4 
15 5 2613

7 1 2024-06-05 10:19:00
7 2 正常订单-账余常规
7 3 
7 4 
7 5 
7 6 
7 7 
7 8 
7 9 
7 10 
7 11 
7 12 
7 13 4560
7 14 
8 1 2024-06-05 10:20:00
8 2 正常订单-账余常规
8 3 
8 4 
8 5 
8 6 
8 7 
8 8 
8 9 
8 10 
8 11 
8 12 
8 13 0
8 14 
9 1 2024-06-14 09:19:00
9 2 正常订单-账余常规
9 3 
9 4 
9 5 
9 6 
9 7 
9 8 
9 9 
9 10 
9 11 
9 12 
9 13 0
9 14 
10 1 2024-06-27 14:11:00
10 2 正常订单-账余常规
10 3 
10 4 
10 5 
10 6 
10 7 
10 8 
10 9 
10 10 
10 11 
10 12 
10 13 9500
10 14 
11 1 2024-06-27 14:11:00
11 2 正常订单-账余常规
11 3 
11 4 
11 5 
11 6 
11 7 
11 8 
11 9 
11 10 
11 11 
11 12 
11 13 0
11 14 
12 1 2024-06-28 10:55:00
12 2 正常订单-账余常规
12 3 
12 4 
12 5 
12 6 
12 7 
12 8 
12 9 
12 10 
12 11 
12 12 
12 13 0
12 14 
13 1 2024-06-28 10:54:00
13 2 正常订单-账余常规
13 3 
13 4 
13 5 
13 6 
13 7 
13 8 
13 9 
13 10 
13 11 
13 12 
13 13 0
13 14 
14 1 2024-06-28 10:54:00
14 2 正常订单-账余常规
14 3 
14 4 
14 5 
14 6 
14 7 
14 8 
14 9 
14 10 
14 11 
14 12 
14 13 0
14 14 
写文件后大小为 2124869
一共有262个工作表，正在处理第260个工作表->青禾信泽C
供应商名称： 青禾信泽品牌咨询管理（深圳）有限公司C
青禾信泽品牌咨询管理（深圳）有限公司

供应商名称： 宁波市鄞州福明花枝俏日用百货商行X
宁波市鄞州福明花枝俏日用百货商行X 既不是招行，也不是工行，将会赋值为空
宁波市鄞州福明花枝俏日用百货商行X 既不是招行，也不是工行，将会赋值为空
=========================将要拷贝到目标文件的内容========================

                 变动时间       变动说明 保证金 工行 招行 订单结案-现金 订单结案-现金核销 订单结案-非现金核销  \
0 2024-06-27 14:14:00  正常订单-账余常规                                          
1 2024-06-27 14:14:00  正常订单-账余常规                                          

  核销-现金核销 核销-非现金核销 出库-现金核销 出库-非现金核销  出库-现金 出库-调换货  
0                                     9750         
1                                        0         
写文件前大小为 2124869
7 1 2024-06-27 14:14:00
7 2 正常订单-账余常规
7 3 
7 4 
7 5 
7 6 
7 7 
7 8 
7 9 
7 10 
7 11 
7 12 
7 13 9750
7 14 
8 1 2024-06-27 14:14:00
8 2 正常订单-账余常规
8 3 
8 4 
8 5 
8 6 
8 7 
8 8 
8 9 
8 10 
8 11 
8 12 
8 13 0
8 14 
写文件后大小为 2124869
一共有262个工作表，正在处理第262个工作表->陕西吉儿美X
供应商名称： 陕西吉儿美化妆品有限公司X
陕西吉儿美化妆品有限公司X 既不是招行，也不是工行，将会赋值为空
陕西吉儿美化妆品有限公司X 既不是招行，也不是工行，将会赋值为空
陕西吉儿美化妆品有限公司X 既不是招行，也不是工行，将会赋值为空
陕西吉儿美化妆品有限公司X 既不是招行，也不是工行，将会赋值为空
陕西吉儿美化妆品有限公司X 既不是招行，也不是工行，将会